<center> <b> <h1> Compute Raw Datasets </h1> </b> </center>

# Index:
### 1. Validation Of Datasets: 
In this part we will verify that we have the same subjects than in the Deep Learning project, and drop the users that are of more. 
### 2. Clean the Datasets:
the features Thickness and Area have in their dataset a column that corresponds at an average, we have to delete it. 
### 3. Join the Lh and Rh in the same Dataframe for each Feature
In This part we will join the Left and tight hemisphere in one dataset for the next computations 
### 4 Save the Datasets whitout class
### 5. Classifing Dataset
We will add the feature Class (1 or -1) to all the 4 datasets 
### 6. Save the Datasets Classified

In [24]:
import pandas as pd

PATH_DATASETS = "../../2_Data_preparation/1_Compute_raw_datasets/Datasets/"
PATH_OUTPUT = "output/"
PATH_STRENGTH = "../../2_Data_preparation/1_Compute_raw_datasets/Datasets/Strength_AgeAdj_labeled.csv"

In [25]:
def readCsvs(lh_or_rh):
    datasets = []
    constat = '.aparc.a2009_'
    features = ['area', 'meancurv', 'thickness', 'volume']
    
    for feature in features:
        df = pd.read_csv(PATH_DATASETS + lh_or_rh + constat + feature + '.csv')
        if lh_or_rh == 'lh':
            df.rename(columns={'lh.aparc.a2009s.' + feature:'Subject'}, inplace=True)
        else:
            df.rename(columns={'rh.aparc.a2009s.' + feature:'Subject'}, inplace=True)
        
        df = df.set_index('Subject')
        datasets.append({'name': feature, 'data': df})
    return datasets
    

In [26]:
def PipelineValidationCleaning(Base_dataset, lh_or_rh_datsets):
    new_lh_or_rh_datsets=[]
    for Data_feature in lh_or_rh_datsets:
        name = Data_feature['name']
        data = Data_feature['data']
        if(Validation(Base_dataset, Data_feature)):
            data1 = CleanSubjects(Base_dataset['data'], data)
            df = {'name': name, 'data': data1}
            if (Validation(df, Base_dataset)):
                new_lh_or_rh_datsets.append(df)
                print( name + " already cleaned" +'\n')
            else:
                print( name + " not was cleaned correctly" +'\n')
        else:
            print("Check the datasets" +'\n')
            print("looks like the subjects not are in the Datasets to work")
    
    return new_lh_or_rh_datsets

In [27]:
def PipelineCleanColumn(lh_or_rh_datsets):
    new_lh_or_rh_datsets=[]
        
    for Data_feature in lh_or_rh_datsets:
        name = Data_feature['name']
        if name == 'thickness' or name == 'area':
            df = CleanLastColumn(Data_feature)
            new_lh_or_rh_datsets.append(df)
        else:
            new_lh_or_rh_datsets.append(Data_feature)
            
    return new_lh_or_rh_datsets

In [28]:
def PipelineMerge(lh_datsets, rh_datsets ):
    new_lh_and_rh_datsets=[]
    
    for i in range(len(lh_datsets)):
        feature = JoinDatasets(lh_datsets[i], rh_datsets[i])
        new_lh_and_rh_datsets.append(feature)
    
    return new_lh_and_rh_datsets

In [29]:
def PipelineMergeClass(lh_datsets, Class ):
    new_lh_and_rh_datsets=[]
    
    nameClass = Class['name']
    
    if nameClass == 'class':
        for i in range(len(lh_datsets)):
            feature = JoinDatasets(lh_datsets[i], Class)
            new_lh_and_rh_datsets.append(feature)
    else:
        print("your are using the wrong function, this is only for merge whit Class Datase (1 ,-1)")
    
    return new_lh_and_rh_datsets

In [30]:
def Validation(Base_dataset, Dataset_to_validate):
    #whit this result we can see that the dataset 3 that is the dataset which we will work, 
    #contains all the subjects that are in the dataset that is shared, dataframe 1.

    if (Base_dataset['data'].index.difference(Dataset_to_validate['data'].index)).size != 0: 
        print("the indices of the " + Base_dataset['name'] + " are not in the indices of "\
              + Dataset_to_validate['name'] + " Dataset")
        return False
    else:
        print("the indices of the " + Base_dataset['name'] + " are in the indices of "\
              + Dataset_to_validate['name'] + " Dataset")
        return True
    

In [31]:
def CleanSubjects(Shared_data_set, Dataset_to_Clean):
    indices_to_delete = Dataset_to_Clean.index.difference(Shared_data_set.index)
    print("this are the indices to delete: " + str(indices_to_delete) + '\n')

    nDataset_to_Clean = Dataset_to_Clean.drop(indices_to_delete, axis=0)
    return nDataset_to_Clean
    

In [32]:
def CleanLastColumn(Dataset_to_clean):
    columns = Dataset_to_clean['data'].columns.tolist()
    df = Dataset_to_clean
    if len(columns) > 74: 
        data = Dataset_to_clean['data'].drop(labels= columns[len(columns)-1], axis=1)
        name = Dataset_to_clean['name']
    
        df = {'name': name, 'data': data}
    else:
        print("You Alreaddy delete the last column, now " + Dataset_to_clean['name'] + " has 74 rois")
        
    return df

In [33]:
def JoinDatasets (Dataset1, Dataset2):
    name_data_set1 = Dataset1['name']
    name_data_set2 = Dataset2['name']
    df_merge_col = pd.merge(Dataset1['data'], Dataset2['data'], on='Subject')
    df = {'name': name_data_set1, 'data' : df_merge_col }
        
    return df

In [34]:
def castfloatarray(array):
    new_array = []
    
    for i in array:
        if type(i) is str:
            x = i.replace(",", ".")
            new_array.append(float(x))
        else:
            pass
    
    return new_array

<center><h2>  1.Validation & cleaning Of Datasets: 
   


In [35]:
#dataset that share machine learning and deep learning
SharedDataSet = {'name':'Shared Dataset', 'data' : pd.read_csv( PATH_DATASETS + "subjects.csv",index_col = 0)}
print(SharedDataSet['data'].shape)
SharedDataSet['data'].head()

(1108, 748)


,b8_lh_NumVert,b8_lh_SurfArea,b8_lh_GrayVol,b8_lh_ThickAvg,b8_lh_ThickStd,b8_lh_MeanCurv,b8_lh_GausCurv,b8_lh_FoldInd,b8_lh_CurvInd,b6_lh_NumVert,...,mw_rh_NumVert,mw_rh_SurfArea,mw_rh_GrayVol,mw_rh_ThickAvg,mw_rh_ThickStd,mw_rh_MeanCurv,mw_rh_GausCurv,mw_rh_FoldInd,mw_rh_CurvInd,class
Subject,,,,,,,,,,,,,,,,,,,,,
585256,3031.0,2015.0,7006.0,2.904,0.553,0.133,0.029,42.0,3.9,6732.0,...,8143.0,5262.0,742.0,0.524,1.014,0.083,0.032,84.0,8.0,-1
585862,3363.0,2168.0,7287.0,2.832,0.463,0.122,0.023,43.0,3.4,7117.0,...,7886.0,4996.0,1009.0,0.608,1.092,0.076,0.024,72.0,5.7,-1
586460,2816.0,1837.0,5913.0,2.687,0.507,0.131,0.027,40.0,3.0,6677.0,...,7077.0,4581.0,1151.0,0.613,1.079,0.089,0.047,82.0,6.6,-1
587664,2991.0,1941.0,7197.0,3.003,0.489,0.138,0.032,49.0,4.0,6763.0,...,7599.0,4722.0,1086.0,0.615,1.100,0.082,0.033,97.0,9.7,-1
588565,2864.0,1845.0,6279.0,2.907,0.451,0.121,0.025,33.0,3.1,5813.0,...,8386.0,5416.0,1337.0,0.609,1.053,0.085,0.026,75.0,8.0,1


In [36]:
# Reading the datasets for hemisphere
Lh_Datasets = readCsvs('lh') #Area,MeanCurv,Thickness,Volume
Rh_Datasets = readCsvs('rh') #Area,MeanCurv,Thickness,Volume

In [37]:
print(Lh_Datasets[0]['data'].shape)
Lh_Datasets[0]['data']

(1113, 75)


,lh_G_and_S_frontomargin_area,lh_G_and_S_occipital_inf_area,lh_G_and_S_paracentral_area,lh_G_and_S_subcentral_area,lh_G_and_S_transv_frontopol_area,lh_G_and_S_cingul-Ant_area,lh_G_and_S_cingul-Mid-Ant_area,lh_G_and_S_cingul-Mid-Post_area,lh_G_cingul-Post-dorsal_area,lh_G_cingul-Post-ventral_area,...,lh_S_pericallosal_area,lh_S_postcentral_area,lh_S_precentral-inf-part_area,lh_S_precentral-sup-part_area,lh_S_suborbital_area,lh_S_subparietal_area,lh_S_temporal_inf_area,lh_S_temporal_sup_area,lh_S_temporal_transverse_area,lh_WhiteSurfArea_area
Subject,,,,,,,,,,,,,,,,,,,,,
100206,1098,1093,820,1156,602,2078,1265,1067,579,383,...,1085,2438,1286,1253,589,875,838,3786,386,104183
100307,627,785,878,1104,521,1678,807,837,381,196,...,758,1503,1070,819,318,718,927,3433,342,79510
100408,1014,1050,790,1213,586,1845,1035,1043,475,205,...,1002,2397,989,1110,435,528,860,3759,357,91914
100610,919,1269,883,813,548,1593,1088,949,410,280,...,821,2214,919,808,490,696,906,3858,325,"93075,8"
101006,687,1059,650,708,316,1120,863,691,306,220,...,481,1822,715,754,436,687,658,2824,284,"71736,4"
101107,789,1115,769,905,427,1528,979,777,423,217,...,592,1681,1208,990,344,550,992,3286,234,"77354,6"
101309,1167,1356,1031,1206,687,1782,1066,938,454,166,...,917,1841,944,999,452,593,760,4476,293,"91687,2"
101410,1108,1275,1344,1183,644,1860,1156,1517,566,415,...,2317,2504,1117,1101,567,807,757,4376,383,102725
101915,743,1310,851,887,467,1442,839,880,350,176,...,739,2168,1232,899,355,826,1034,3931,265,"81488,9"


In [38]:
Lh_Datasets = PipelineValidationCleaning(SharedDataSet, Lh_Datasets)

the indices of the Shared Dataset are in the indices of area Dataset
this are the indices to delete: Int64Index([114419, 146937, 153732, 164636, 206222], dtype='int64', name='Subject')

the indices of the area are in the indices of Shared Dataset Dataset
area already cleaned

the indices of the Shared Dataset are in the indices of meancurv Dataset
this are the indices to delete: Int64Index([114419, 146937, 153732, 164636, 206222], dtype='int64', name='Subject')

the indices of the meancurv are in the indices of Shared Dataset Dataset
meancurv already cleaned

the indices of the Shared Dataset are in the indices of thickness Dataset
this are the indices to delete: Int64Index([114419, 146937, 153732, 164636, 206222], dtype='int64', name='Subject')

the indices of the thickness are in the indices of Shared Dataset Dataset
thickness already cleaned

the indices of the Shared Dataset are in the indices of volume Dataset
this are the indices to delete: Int64Index([114419, 146937, 153732, 164

In [39]:
Rh_Datasets = PipelineValidationCleaning(SharedDataSet, Rh_Datasets)

the indices of the Shared Dataset are in the indices of area Dataset
this are the indices to delete: Int64Index([114419, 146937, 153732, 164636, 206222], dtype='int64', name='Subject')

the indices of the area are in the indices of Shared Dataset Dataset
area already cleaned

the indices of the Shared Dataset are in the indices of meancurv Dataset
this are the indices to delete: Int64Index([114419, 146937, 153732, 164636, 206222], dtype='int64', name='Subject')

the indices of the meancurv are in the indices of Shared Dataset Dataset
meancurv already cleaned

the indices of the Shared Dataset are in the indices of thickness Dataset
this are the indices to delete: Int64Index([114419, 146937, 153732, 164636, 206222], dtype='int64', name='Subject')

the indices of the thickness are in the indices of Shared Dataset Dataset
thickness already cleaned

the indices of the Shared Dataset are in the indices of volume Dataset
this are the indices to delete: Int64Index([114419, 146937, 153732, 164

In [40]:
# whit this you can compare that the number of subjects has been reduced from 1113 to 1108
print(Lh_Datasets[0]['data'].shape)
Lh_Datasets[0]['data'].head()

(1108, 75)


,lh_G_and_S_frontomargin_area,lh_G_and_S_occipital_inf_area,lh_G_and_S_paracentral_area,lh_G_and_S_subcentral_area,lh_G_and_S_transv_frontopol_area,lh_G_and_S_cingul-Ant_area,lh_G_and_S_cingul-Mid-Ant_area,lh_G_and_S_cingul-Mid-Post_area,lh_G_cingul-Post-dorsal_area,lh_G_cingul-Post-ventral_area,...,lh_S_pericallosal_area,lh_S_postcentral_area,lh_S_precentral-inf-part_area,lh_S_precentral-sup-part_area,lh_S_suborbital_area,lh_S_subparietal_area,lh_S_temporal_inf_area,lh_S_temporal_sup_area,lh_S_temporal_transverse_area,lh_WhiteSurfArea_area
Subject,,,,,,,,,,,,,,,,,,,,,
100206,1098,1093,820,1156,602,2078,1265,1067,579,383,...,1085,2438,1286,1253,589,875,838,3786,386,104183
100307,627,785,878,1104,521,1678,807,837,381,196,...,758,1503,1070,819,318,718,927,3433,342,79510
100408,1014,1050,790,1213,586,1845,1035,1043,475,205,...,1002,2397,989,1110,435,528,860,3759,357,91914
100610,919,1269,883,813,548,1593,1088,949,410,280,...,821,2214,919,808,490,696,906,3858,325,"93075,8"
101006,687,1059,650,708,316,1120,863,691,306,220,...,481,1822,715,754,436,687,658,2824,284,"71736,4"


<center><h2>  2. cleaning last column: 

In [41]:
Lh_Datasets = PipelineCleanColumn(Lh_Datasets)

In [42]:
Rh_Datasets = PipelineCleanColumn(Rh_Datasets)

In [43]:
print(Lh_Datasets[0]['data'].shape)
Lh_Datasets[0]['data'].head()

(1108, 74)


,lh_G_and_S_frontomargin_area,lh_G_and_S_occipital_inf_area,lh_G_and_S_paracentral_area,lh_G_and_S_subcentral_area,lh_G_and_S_transv_frontopol_area,lh_G_and_S_cingul-Ant_area,lh_G_and_S_cingul-Mid-Ant_area,lh_G_and_S_cingul-Mid-Post_area,lh_G_cingul-Post-dorsal_area,lh_G_cingul-Post-ventral_area,...,lh_S_parieto_occipital_area,lh_S_pericallosal_area,lh_S_postcentral_area,lh_S_precentral-inf-part_area,lh_S_precentral-sup-part_area,lh_S_suborbital_area,lh_S_subparietal_area,lh_S_temporal_inf_area,lh_S_temporal_sup_area,lh_S_temporal_transverse_area
Subject,,,,,,,,,,,,,,,,,,,,,
100206,1098,1093,820,1156,602,2078,1265,1067,579,383,...,2066,1085,2438,1286,1253,589,875,838,3786,386
100307,627,785,878,1104,521,1678,807,837,381,196,...,1458,758,1503,1070,819,318,718,927,3433,342
100408,1014,1050,790,1213,586,1845,1035,1043,475,205,...,1930,1002,2397,989,1110,435,528,860,3759,357
100610,919,1269,883,813,548,1593,1088,949,410,280,...,1614,821,2214,919,808,490,696,906,3858,325
101006,687,1059,650,708,316,1120,863,691,306,220,...,1419,481,1822,715,754,436,687,658,2824,284


<center><h2>  3. Join Datasets Lh and rh in one and change the (,) for (.) in meancurv and thickness: 

In [44]:
lh_and_rh_Dataset = PipelineMerge(Lh_Datasets, Rh_Datasets)
print(lh_and_rh_Dataset[1]['data'].shape)
lh_and_rh_Dataset[1]['data'].head()


(1108, 148)


,lh_G_and_S_frontomargin_meancurv,lh_G_and_S_occipital_inf_meancurv,lh_G_and_S_paracentral_meancurv,lh_G_and_S_subcentral_meancurv,lh_G_and_S_transv_frontopol_meancurv,lh_G_and_S_cingul-Ant_meancurv,lh_G_and_S_cingul-Mid-Ant_meancurv,lh_G_and_S_cingul-Mid-Post_meancurv,lh_G_cingul-Post-dorsal_meancurv,lh_G_cingul-Post-ventral_meancurv,...,rh_S_parieto_occipital_meancurv,rh_S_pericallosal_meancurv,rh_S_postcentral_meancurv,rh_S_precentral-inf-part_meancurv,rh_S_precentral-sup-part_meancurv,rh_S_suborbital_meancurv,rh_S_subparietal_meancurv,rh_S_temporal_inf_meancurv,rh_S_temporal_sup_meancurv,rh_S_temporal_transverse_meancurv
Subject,,,,,,,,,,,,,,,,,,,,,
100206,"0,177","0,165","0,144","0,178","0,198","0,16","0,17","0,171","0,219","0,217",...,"0,158","0,175","0,128","0,145","0,137","0,152","0,159","0,136","0,136","0,156"
100307,"0,173","0,15","0,132","0,183","0,218","0,175","0,157","0,16","0,243","0,198",...,"0,148","0,188","0,152","0,123","0,125","0,157","0,152","0,144","0,137","0,147"
100408,"0,167","0,173","0,137","0,166","0,18","0,169","0,169","0,179","0,241","0,178",...,"0,148","0,173","0,132","0,142","0,116","0,139","0,177","0,131","0,131","0,137"
100610,"0,138","0,153","0,116","0,139","0,157","0,133","0,151","0,154","0,196","0,163",...,"0,133","0,161","0,125","0,115","0,119","0,139","0,131","0,114","0,122","0,145"
101006,"0,152","0,155","0,121","0,153","0,179","0,157","0,138","0,131","0,202","0,175",...,"0,137","0,122","0,109","0,118","0,112","0,166","0,16","0,135","0,126","0,135"


In [45]:
new_lh_and_rh_Dataset = []

for feature in lh_and_rh_Dataset: 
    data = feature['data']
    name = feature['name']
    if name == 'meancurv' or name == 'thickness':
        new_df = data.copy()
        for column in data.columns.values:
            x = castfloatarray(data[column].values)
            conter = 0 
            for index in new_df.index.values:
                new_df.at[index, column] = x[conter]
                conter += 1
        df = {'name': name, 'data': new_df} 
        new_lh_and_rh_Dataset.append(df)
    else:
        new_lh_and_rh_Dataset.append(feature)

lh_and_rh_Dataset = new_lh_and_rh_Dataset

In [46]:
lh_and_rh_Dataset[1]['data'].head()

,lh_G_and_S_frontomargin_meancurv,lh_G_and_S_occipital_inf_meancurv,lh_G_and_S_paracentral_meancurv,lh_G_and_S_subcentral_meancurv,lh_G_and_S_transv_frontopol_meancurv,lh_G_and_S_cingul-Ant_meancurv,lh_G_and_S_cingul-Mid-Ant_meancurv,lh_G_and_S_cingul-Mid-Post_meancurv,lh_G_cingul-Post-dorsal_meancurv,lh_G_cingul-Post-ventral_meancurv,...,rh_S_parieto_occipital_meancurv,rh_S_pericallosal_meancurv,rh_S_postcentral_meancurv,rh_S_precentral-inf-part_meancurv,rh_S_precentral-sup-part_meancurv,rh_S_suborbital_meancurv,rh_S_subparietal_meancurv,rh_S_temporal_inf_meancurv,rh_S_temporal_sup_meancurv,rh_S_temporal_transverse_meancurv
Subject,,,,,,,,,,,,,,,,,,,,,
100206,0.177,0.165,0.144,0.178,0.198,0.16,0.17,0.171,0.219,0.217,...,0.158,0.175,0.128,0.145,0.137,0.152,0.159,0.136,0.136,0.156
100307,0.173,0.15,0.132,0.183,0.218,0.175,0.157,0.16,0.243,0.198,...,0.148,0.188,0.152,0.123,0.125,0.157,0.152,0.144,0.137,0.147
100408,0.167,0.173,0.137,0.166,0.18,0.169,0.169,0.179,0.241,0.178,...,0.148,0.173,0.132,0.142,0.116,0.139,0.177,0.131,0.131,0.137
100610,0.138,0.153,0.116,0.139,0.157,0.133,0.151,0.154,0.196,0.163,...,0.133,0.161,0.125,0.115,0.119,0.139,0.131,0.114,0.122,0.145
101006,0.152,0.155,0.121,0.153,0.179,0.157,0.138,0.131,0.202,0.175,...,0.137,0.122,0.109,0.118,0.112,0.166,0.16,0.135,0.126,0.135


<center><h2>  4. Save the new Datasets: 

In [47]:
for feature in lh_and_rh_Dataset:
    name = feature['name']
    data = feature['data']
    
    data.to_csv(PATH_OUTPUT + name + '.csv')

<center><h2>  5. add Class to the Datasets: 

In [48]:
Class_dataset = {'name': 'class', 'data': pd.read_csv(PATH_STRENGTH, index_col=0)}
print(Class_dataset['data'].shape)
Class_dataset = {'name': 'class', 'data':Class_dataset['data'].drop(['Strength_AgeAdj'], axis=1)}
print(Class_dataset['data'].shape)
Class_dataset['data'].head()

(1205, 2)
(1205, 1)


,class
Subject,
100004,1.0
100206,1.0
100307,-1.0
100408,1.0
100610,1.0


In [49]:
lh_and_rh_Dataset = PipelineMergeClass(lh_and_rh_Dataset, Class_dataset)
print(lh_and_rh_Dataset[0]['data'].shape)
lh_and_rh_Dataset[0]['data'].head()

(1108, 149)


,lh_G_and_S_frontomargin_area,lh_G_and_S_occipital_inf_area,lh_G_and_S_paracentral_area,lh_G_and_S_subcentral_area,lh_G_and_S_transv_frontopol_area,lh_G_and_S_cingul-Ant_area,lh_G_and_S_cingul-Mid-Ant_area,lh_G_and_S_cingul-Mid-Post_area,lh_G_cingul-Post-dorsal_area,lh_G_cingul-Post-ventral_area,...,rh_S_pericallosal_area,rh_S_postcentral_area,rh_S_precentral-inf-part_area,rh_S_precentral-sup-part_area,rh_S_suborbital_area,rh_S_subparietal_area,rh_S_temporal_inf_area,rh_S_temporal_sup_area,rh_S_temporal_transverse_area,class
Subject,,,,,,,,,,,,,,,,,,,,,
100206,1098,1093,820,1156,602,2078,1265,1067,579,383,...,1388,1897,1485,1002,308,1154,944,5652,267,1.0
100307,627,785,878,1104,521,1678,807,837,381,196,...,775,1482,928,1008,115,601,991,3606,256,-1.0
100408,1014,1050,790,1213,586,1845,1035,1043,475,205,...,1173,1913,1415,825,181,921,1063,4289,251,1.0
100610,919,1269,883,813,548,1593,1088,949,410,280,...,1128,1331,1061,1191,155,609,1089,4572,170,1.0
101006,687,1059,650,708,316,1120,863,691,306,220,...,2323,1699,654,839,202,717,888,3446,199,1.0


<center><h2>  6. Save the new Datasets Classified: 

In [50]:
for feature in lh_and_rh_Dataset:
    name = feature['name']
    data = feature['data']
    
    data.to_csv(PATH_OUTPUT + name + '_Classified.csv')